In [5]:
import pandas as pd
import numpy as np

# demand-forecasting-kernels-only

In [6]:
train = pd.read_csv(r"data/demand-forecasting-kernels-only/train.csv")
train

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


In [7]:
train.date.min(),train.date.max()

('2013-01-01', '2017-12-31')

In [8]:
test = pd.read_csv(r"data/demand-forecasting-kernels-only/test.csv")
test

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1
...,...,...,...,...
44995,44995,2018-03-27,10,50
44996,44996,2018-03-28,10,50
44997,44997,2018-03-29,10,50
44998,44998,2018-03-30,10,50


In [9]:
test_min_date, test_max_date = test.date.unique().min(),test.date.unique().max()
test_min_date, test_max_date

('2018-01-01', '2018-03-31')

In [10]:
df = pd.concat([train, test], axis=0)
df0 = df.copy(deep=True)
df

,date,store,item,sales,id
0,2013-01-01,1,1,13.0,NaN
1,2013-01-02,1,1,11.0,NaN
2,2013-01-03,1,1,14.0,NaN
3,2013-01-04,1,1,13.0,NaN
4,2013-01-05,1,1,10.0,NaN
...,...,...,...,...,...
44995,2018-03-27,10,50,NaN,44995.0
44996,2018-03-28,10,50,NaN,44996.0
44997,2018-03-29,10,50,NaN,44997.0
44998,2018-03-30,10,50,NaN,44998.0


# Utils functions and user configurations

In [111]:
from itertools import product
from functools import reduce # Valid in Python 2.6+, required in Python 3
import operator
import sys

def print_func_name():
    the_name = sys._getframe(1).f_code.co_name
    return print(f'{the_name}()')

def move_cols_to_first(the_df, first_cols: list):
    print_func_name()
    latter_cols = the_df.columns[~the_df.columns.isin(first_cols)]
    the_df = pd.concat([the_df[first_cols], the_df[latter_cols]],axis=1)
    return the_df

def add_indexes_col(the_df):
    print_func_name()
    indexes_col = "_".join(index_cols)
    the_df = the_df.copy(deep=True)
    the_df[indexes_col] = the_df[index_cols[0]].astype(str)
    if len(indexes_col)>1:
        for col in index_cols[1:]:
            the_df[indexes_col] = the_df[indexes_col] + "_" + the_df[col].astype(str)
    the_df = move_cols_to_first(the_df, [time_col,indexes_col]+index_cols)
    return the_df, indexes_col

def is_full_time_df(the_df):
    print_func_name()
    is_full_time = len(the_df) == reduce(operator.mul, the_df[index_cols+[time_col]].nunique())
    return is_full_time

def is_target_clean(the_df):
    print_func_name()
    return the_df[target].isna().sum() == 0

def target_fillna(the_df):
    print_func_name()
    assert target in the_df
    the_df = the_df.fillna({target:target_fillna_value})
    assert is_target_clean(the_df)
    return the_df

def target_dropna(the_df):
    print_func_name()
    the_df = the_df.dropna(subset=[target])
    assert the_df[target].isna().sum() == 0

def target_fix_na(the_df):
    print_func_name()
    if target_na_decision=='fill':
        the_df = target_fillna(the_df)
    elif target_na_decision=='drop':
        the_df = target_dropna(the_df)
    return the_df

def recreate_index_cols_from_indexes_col(the_df):
    the_indexes_col = "_".join(index_cols)
    assert the_indexes_col in the_df
    index_cols_df = the_df[the_indexes_col].str.split("_", expand=True).reset_index(drop=True)
    index_cols_df.columns = index_cols
    the_df = pd.concat([the_df[time_col, the_indexes_col], index_cols_df, the_df.drop(index_cols+[time_col, the_indexes_col], errors='ignore')], axis=1).reset_index(drop=True)
    return the_df

def add_missing_dates_fill_target_zero(the_df):
    print_func_name()
    if is_full_time_df(the_df):
        return the_df
    # Make sure there's one index col
    the_indexes_col = "_".join(index_cols)
    if not the_indexes_col in the_df:
        the_df, the_indexes_col = add_indexes_col(the_df)
    # The logic
    the_df_full_date_indexes = pd.DataFrame(product(the_df[time_col].drop_duplicates(), the_df[the_indexes_col].drop_duplicates()), columns=[time_col, the_indexes_col]).dropna()
    the_df = the_df.merge(the_df_full_date_indexes, how='right')
    ## fillna's
    the_df = recreate_index_cols_from_indexes_col(the_df)
    print(the_df)
    the_df = target_fillna(the_df)
    assert is_full_time_df(the_df)
    return the_df

# user configurations
time_col = 'date'
index_cols = ['store','item']
target = 'sales'
df = df0.copy(deep=True)
df['storeXitem'] = df.store * df.item
df['storeX5'] = df.store*5
df['itemX5'] = df.item*5
features_orig = df.columns[~df.columns.isin(index_cols+[time_col, target])]
target_na_decision = 'fill' # or drop
target_fillna_value = 0

# pipeline
#df = target_fix_na(df)

df = target_fix_na(df)
df = add_missing_dates_fill_target_zero(df[1:])
df

target_fix_na()
target_fillna()
is_target_clean()
add_missing_dates_fill_target_zero()
is_full_time_df()
add_indexes_col()
move_cols_to_first()


KeyError: ('date', 'store_item')

In [105]:
index_cols+[time_col]

['store', 'item', 'date']

In [106]:
df

,date,store,item,date,store_item,store,item,sales,id,storeXitem,storeX5,itemX5
0,2013-01-02,1,1,2013-01-02,1_1,1.0,1.0,11.0,NaN,1.0,5.0,5.0
1,2013-01-03,1,1,2013-01-03,1_1,1.0,1.0,14.0,NaN,1.0,5.0,5.0
2,2013-01-04,1,1,2013-01-04,1_1,1.0,1.0,13.0,NaN,1.0,5.0,5.0
3,2013-01-05,1,1,2013-01-05,1_1,1.0,1.0,10.0,NaN,1.0,5.0,5.0
4,2013-01-06,1,1,2013-01-06,1_1,1.0,1.0,12.0,NaN,1.0,5.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
957995,2018-03-28,10,50,2018-03-28,10_50,10.0,50.0,0.0,44996.0,500.0,50.0,250.0
957996,2018-03-29,10,50,2018-03-29,10_50,10.0,50.0,0.0,44997.0,500.0,50.0,250.0
957997,2018-03-30,10,50,2018-03-30,10_50,10.0,50.0,0.0,44998.0,500.0,50.0,250.0
957998,2018-03-31,10,50,2018-03-31,10_50,10.0,50.0,0.0,44999.0,500.0,50.0,250.0


In [103]:
reduce(operator.mul, df[index_cols+[time_col]].nunique())

917764000000

In [36]:
df = df0.copy(deep=True)
    # Maake sure there's one index col
the_indexes_col = "_".join(index_cols)
if not the_indexes_col in df:
    the_df, the_indexes_col = add_concat_index_cols(df)
# The logic
the_df_full_date_indexes = pd.DataFrame(product(df[time_col].drop_duplicates(), the_df[the_indexes_col].drop_duplicates()), columns=[time_col, the_indexes_col]).dropna()


add_concat_index_cols()
move_cols_to_first()


## find fixed in time features

In [46]:
n_values_feature_per_index = df.groupby(index_cols)[features_orig].nunique(dropna=False).nunique(dropna=False)
n_values_feature_per_index

id            1
storeXitem    1
dtype: int64

In [67]:
def get_features_time_dependence(the_df):
    is_fixed_features_df = (the_df.groupby(index_cols)[features_orig].nunique(dropna=False) == 1).all()
    the_time_fixed_features = is_fixed_features_df[is_fixed_features_df==True].index
    the_time_variant_features = is_fixed_features_df[is_fixed_features_df==False].index
    return the_time_fixed_features, the_time_variant_features

def time_fixed_features_fillna(the_df):


time_fixed_features, time_variant_features = get_features_time_dependence(df)
time_variant_features

Index(['id'], dtype='object')

In [76]:
time_fixed_features_fill_values = df.groupby(index_cols)[time_fixed_features].last().reset_index()
df.merge(time_fixed_features_fill_values, on=index_cols, how='left')

,date,store_item,store,item,sales,id,storeXitem_x,storeX5_x,itemX5_x,storeXitem_y,storeX5_y,itemX5_y
0,2013-01-02,1_1,1.0,1.0,11.0,NaN,1.0,5.0,5.0,1.0,5.0,5.0
1,2013-01-03,1_1,1.0,1.0,14.0,NaN,1.0,5.0,5.0,1.0,5.0,5.0
2,2013-01-04,1_1,1.0,1.0,13.0,NaN,1.0,5.0,5.0,1.0,5.0,5.0
3,2013-01-05,1_1,1.0,1.0,10.0,NaN,1.0,5.0,5.0,1.0,5.0,5.0
4,2013-01-06,1_1,1.0,1.0,12.0,NaN,1.0,5.0,5.0,1.0,5.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
957995,2018-03-28,10_50,10.0,50.0,0.0,44996.0,500.0,50.0,250.0,500.0,50.0,250.0
957996,2018-03-29,10_50,10.0,50.0,0.0,44997.0,500.0,50.0,250.0,500.0,50.0,250.0
957997,2018-03-30,10_50,10.0,50.0,0.0,44998.0,500.0,50.0,250.0,500.0,50.0,250.0
957998,2018-03-31,10_50,10.0,50.0,0.0,44999.0,500.0,50.0,250.0,500.0,50.0,250.0


In [119]:
df = df0.copy(deep=True)
df, indexes_col = add_indexes_col(df)
df

add_indexes_col()
move_cols_to_first()


,date,store_item,store,item,sales,id
0,2013-01-01,1_1,1,1,13.0,NaN
1,2013-01-02,1_1,1,1,11.0,NaN
2,2013-01-03,1_1,1,1,14.0,NaN
3,2013-01-04,1_1,1,1,13.0,NaN
4,2013-01-05,1_1,1,1,10.0,NaN
...,...,...,...,...,...,...
44995,2018-03-27,10_50,10,50,NaN,44995.0
44996,2018-03-28,10_50,10,50,NaN,44996.0
44997,2018-03-29,10_50,10,50,NaN,44997.0
44998,2018-03-30,10_50,10,50,NaN,44998.0


In [120]:
recreate_index_cols_from_indexes_col(df)

KeyError: ('date', 'store_item')

In [87]:
def recreate_index_cols_from_indexes_col(the_df):
    the_indexes_col = "_".join(index_cols)
    assert the_indexes_col in the_df
    index_cols_df = the_df[the_indexes_col].str.split("_", expand=True).reset_index(drop=True)
    index_cols_df.columns = index_cols
    the_df = pd.concat([the_df[time_col, the_indexes_col], index_cols_df, the_df.drop(index_cols+[time_col, the_indexes_col], errors='ignore')], axis=1).reset_index(drop=True)
    return the_df

,date,store,item,date,store_item,store,item,sales,id,storeXitem,storeX5,itemX5
0,2013-01-02,1,1,2013-01-02,1_1,1.0,1.0,11.0,NaN,1.0,5.0,5.0
1,2013-01-03,1,1,2013-01-03,1_1,1.0,1.0,14.0,NaN,1.0,5.0,5.0
2,2013-01-04,1,1,2013-01-04,1_1,1.0,1.0,13.0,NaN,1.0,5.0,5.0
3,2013-01-05,1,1,2013-01-05,1_1,1.0,1.0,10.0,NaN,1.0,5.0,5.0
4,2013-01-06,1,1,2013-01-06,1_1,1.0,1.0,12.0,NaN,1.0,5.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
957995,2018-03-28,10,50,2018-03-28,10_50,10.0,50.0,0.0,44996.0,500.0,50.0,250.0
957996,2018-03-29,10,50,2018-03-29,10_50,10.0,50.0,0.0,44997.0,500.0,50.0,250.0
957997,2018-03-30,10,50,2018-03-30,10_50,10.0,50.0,0.0,44998.0,500.0,50.0,250.0
957998,2018-03-31,10,50,2018-03-31,10_50,10.0,50.0,0.0,44999.0,500.0,50.0,250.0


In [82]:
temp = df[the_indexes_col].str.split("_", expand=True)
temp.columns = index_cols
temp

,store,item
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
957995,10,50
957996,10,50
957997,10,50
957998,10,50


In [65]:
the_df = the_df.merge(the_df_full_date_indexes, how='right')
the_df = target_fillna(the_df)

target_fillna()
is_target_clean()


# handle features NA

# Target encoding categoricals

# Feature Engineering

# TODO
## create a unified time col from multiple time cols